## 2.2 Hierarchical Data and the Closure Property

An operation for combining data objects satisfies the **Closure property** if the result of 
combining things with that operation can themselves be combined using the same operation.

Closure is the key to power in any means of combination, because it permits us to create 
hierarchical structures.

> It is better to have 100 functions operate on one data structure than to have 10 functions operate on 10 data structures.

In [2]:
(defn cons [x y]
  (fn [m] (m x y)))
(defn car [lst]
  (lst (fn [x y] x)))
(defn cdr [lst]
  (lst (fn [x y] y)))

#'user/cdr

In [3]:
(cons (cons 1 2) (cons 3 4))

#function[user/cons/fn--10843]

In [7]:
; Exercise 2.17
(defn last-pair [lst] (last lst))
(last-pair [1 2 3])

; Exercise 2.18
(defn rev [lst]
   (loop [c lst 
          acc '()]
     (if c (recur (next c)
                  (conj acc (first c)))  
         acc)))
(rev [1, 2, 3, [1,2]])


; Exercise 2.19 
(def us-coins [50, 25, 10, 5, 1])
(def uk-coins [100, 50, 20, 10, 5, 2, 1, 0.5])
(def no-more? nil?)
(def except-first-denomination next)
(def first-denomination first)
(defn cc [amount coin-values]
  (cond (= amount 0) 1
        (or (< amount 0) (no-more? coin-values)) 0
        :else (+ (cc amount 
                     (except-first-denomination coin-values))
                 (cc (- amount (first-denomination coin-values))
                     coin-values))))

(cc 100 us-coins)
; 292

; Exercise 2.20 
(defn same-parity [x & xs]
  cons x (filter (if (even? x) even? odd?) xs))
(same-parity 1 2 3 4 5)
(same-parity 2 3 4 5 6)

(4 6)

**Exercise 2.22**

The first solution, order is revered. It can be found by using the subsistute model. The second solution wont work neither, as the result will be in this pattern `(cons (cons ...) ..)`

In [8]:
; Exercise 2.23
(defn for-each-lazy [f coll]
  (map f coll))
(defn for-each-strict [f coll]
  (doall (for-each-lazy f coll)))

(for-each-strict (fn [x] (println x)) '(1 2 3))

1
2
3


(nil nil nil)

As we already see that **Data Abstraction** give us ability to work on the data without knowing its implementation, and the also gives us ability to use different implementation/representation of the data. 

Now, we are going through a new techneque **Conventioanl Interfaces**.

The value of expressing programs as sequence operations is that this helps us make program designs that are modular.

> We can encourage modular design by providing a library of standard components together with a conventional interface for connecting the components in flexible ways.

**Sequences** serves as a conventioanl interface that permist us to combine processing modules. By uniforming the representation to a sequence, we limit our program to a small set of operations, such as map, filter, reduce.

In this section, Sequency is represented as list, but we could use other reprensentation.

Bascally, here we are talking about map, filter and fold functions. And in the end, it is all about fold. techniquely, we can make any data type foldable. 

In [4]:
(comment
  "
  2.2.2 Hierarchical Sturctures
  Map, fold, filter still powerful in nested structures, like trees and graph.
  ")

(defn count-leaves [x]
  (cond (nil? x) 0 
        (not (list? x)) 1))

(count-leaves '()) 


; Exercise 2.33 
(defn map' [f xs]
  (reduce #(concat %1 (list (f %2))) '() xs))
(defn append' [coll1 coll2]
  (reduce conj coll2 (reverse coll1))) 
(defn length' [coll]
  (reduce (fn [acc x] (+ 1 acc)) 0 coll))

(map' #(* 2 %1) '(1 2 3))
(append' '(1 2 3) '(1 2 3))
(length' '(1 2))

; Exercise 2.34
(defn foldr [func init coll]
  (if (nil? coll) init 
    (func (first coll)
          (foldr func init (next coll)))))

(defn horner-eval [x coefficient-sequence]
  (foldr #(+ (* %2 x) %1) 0 coefficient-sequence))

(horner-eval 2 '(1 3 0 5 0 1))

; Exercise 2.36 
(defn accumulate [op init coll]
  (if (nil? coll) init
    ((println coll)
     (op (first coll)
         (accumulate op init (next coll))))))
(defn accumulate-n [op init coll]
  (if (nil? (first coll))
    nil 
    (cons (accumulate op init (map first coll))
          (accumulate-n op init (map next coll)))))

(def s1 '( '(1 2 3) '(4 5 6)))

; Exercise 2.37 
(defn dot-product [v w]
  (reduce + 0 (map * v w)))
(defn matrix-*-vector [m v]
  (map (partial dot-product v) m))
(defn transpose [mat] 
  (accumulate-n cons nil mat))
(defn matrix-*-matrix [m n]
  (let [cols (transpose n)]
    (map (fn [row] (map #(dot-product row %) cols)) m)))
  
(comment 
  "
  Nested Mappings
  ")

; Exercise 2.40 
(defn unique-pairs [n]
  (mapcat (fn [x]
            (map (partial list x) (range (inc x) (inc n))))
          (range 1 n)))

(unique-pairs 5)
; ((1 2) (1 3) (1 4) (1 5) (2 3) (2 4) (2 5) (3 4) (3 5) (4 5))

; Exercise 2.41 
(defn triples [n]
  (for [x (range 1 (dec n))
        y (range (inc x) n)
        z (range (inc y) (inc n))]
    [x y z]))

; Exercise 2.42 Eight-queens
(def empty-board '())

(defn safe? [positions]
  (let [[queen-pos & left] positions
        k (count positions)
        diags-up (map - left (range 1 (inc k)))
        diags-down (map + left (range 1 (inc k)))]
    (empty? (filter (partial = queen-pos) (concat diags-up diags-down left)))))
  
(defn queens [board-size]
  ((fn queen-cols [k]
     (if (zero? k)
       (list empty-board)
       (for [rest-of-queens (queen-cols (dec k))
             new-row (range 1 (inc board-size))
             :let [new-cols (cons new-row rest-of-queens)]
             :when (safe? new-cols)]
         new-cols)))
   board-size))

(queens 6)

((5 3 1 6 4 2) (4 1 5 2 6 3) (3 6 2 5 1 4) (2 4 6 1 3 5))

**The Picture Language**

Describe the language by three aspects:
- primitives
- means of combination
- means of abstraction

The closure of data under the means of combination is crucial to the ability to create complex structures while using only a few operations.

**Levels of language for robust design**

Or using layers to design system. Each layer has a "language" which has: primitives, means of combination, means of abstraction. This kind of system is easy to modify. In general, each level of a stratified design provides a different vocabulary for express- ing the characteristics of the system, and a different kind of ability to change it.